In [ ]:
%config IPCompleter.greedy=True

# Import the dependencies.
import json
import re
import time

import numpy as np
import pandas as pd
import requests

In [ ]:
# The request files to verify its structure.
base_url = "https://stats.nba.com/stats/"

headers = {
    "Host": "stats.nba.com",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0",
    "Accept": "application/json, text/plain, */*",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br",
    "x-nba-stats-origin": "stats",
    "x-nba-stats-token": "true",
    "Connection": "keep-alive",
    "Referer": "https://stats.nba.com/",
    "Pragma": "no-cache",
    "Cache-Control": "no-cache",
}

measure_type = {"base": "Base", "advanced": "Advanced"}

stat_modes_depth = {
    "Per100Possessions": "Per100Possessions",
    "Totals": "Totals",
    "Per36": "Per36",
    "PerGame": "PerGame",
}

season_id = "2019-20"
team_id = 1610612737
per_mode = "PerGame"

season_list = [
    "1996-97",
    "1997-98",
    "1998-99",
    "1999-00",
    "2000-01",
    "2001-02",
    "2002-03",
    "2003-04",
    "2004-05",
    "2005-06",
    "2006-07",
    "2007-08",
    "2008-09",
    "2009-10",
    "2010-11",
    "2011-12",
    "2012-13",
    "2013-14",
    "2014-15",
    "2015-16",
    "2016-17",
    "2017-18",
    "2018-19",
    "2019-20",
]

list_of_nbastats_apis = {
    # base_url, per-mode, season
    "player_general_traditional_totals": f"{base_url}leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode={per_mode}&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season={season_id}&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision=&Weight=",
    # base_url, per-mode, season
    "player_general_advanced_totals": f"{base_url}leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Advanced&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode={per_mode}&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season={season_id}&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision=&Weight=",
    # base_url, per-mode, season
    "player_bios": f"{base_url}leaguedashplayerbiostats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PerMode={per_mode}&Period=0&PlayerExperience=&PlayerPosition=&Season={season_id}&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight=",
    # base_url, season, but look into PlayerOrTeam
    "player_game_logs": f"{base_url}leaguegamelog?Counter=1000&DateFrom=&DateTo=&Direction=DESC&LeagueID=00&PlayerOrTeam=P&Season={season_id}&SeasonType=Regular+Season&Sorter=DATE",
    # base_url, season, but look into PlayerOrTeam
    "team_game_logs": f"{base_url}leaguegamelog?Counter=1000&DateFrom=&DateTo=&Direction=DESC&LeagueID=00&PlayerOrTeam=T&Season={season_id}&SeasonType=Regular+Season&Sorter=DATE",
    # base_url, per-mode, season
    "team_general_traditional_stats": f"{base_url}leaguedashteamstats?DateFrom=&DateTo=&GameScope=&GameSegment=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PaceAdjust=N&PerMode={per_mode}&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season={season_id}&SeasonSegment=&SeasonType=Regular+Season&StarterBench=&VsConference=&VsDivision=",
    # base_url, per-mode, season
    "team_general_advanced_stats": f"{base_url}leaguedashteamstats?Conference=&DateFrom=&DateTo=&Division=&GameScope=&GameSegment=&LastNGames=0&LeagueID=00&Location=&MeasureType=Advanced&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode={per_mode}&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season={season_id}&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision=",
    # base_url, per-mode, season, team
    "players_by_team": f"{base_url}teamplayerdashboard?DateFrom=&DateTo=&GameSegment=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PaceAdjust=N&PerMode={per_mode}&Period=0&PlusMinus=N&Rank=N&Season={season_id}&SeasonSegment=&SeasonType=Regular+Season&TeamID={team_id}&VsConference=&VsDivision=",
    # base_url, season
    "player_abbr": f"{base_url}commonallplayers?IsOnlyCurrentSeason=0&LeagueID=00&Season={season_id}",
    # base_url, team, season
    "team_abbr": f"{base_url}teaminfocommon?LeagueID=00&SeasonType=Regular+Season&TeamID={team_id}&Season={season_id}",
}

# # We want to append all the headers from the different apis to a master list
# master_header_list = []

# for key, url in list_of_nbastats_apis.items():
#     # json response
#     response = requests.get(url=url, headers=headers).json()
#     # pretty printing data
#     # pretty_data = json.dumps(response, indent=4)
#     # print(pretty_data)
#     # pulling the headers we want to experiment with
#     stat_header = response["resultSets"][0]["headers"]
#     print(stat_header)
#     # pulling just the data we want
#     body_data = response["resultSets"][0]["rowSet"][0]
#     print(body_data)
#     # Append the header list to the master header list
#     master_header_list.append(dict({f"{key}": [x.lower() for x in stat_header]}))

# #print(f"{master_header_list}")

# I want to just download the CSV version of Player's General Traditional stats
# https://www.nba.com/stats/players/traditional/?sort=PTS&dir=-1&Season=2019-20&SeasonType=Regular%20Season

response = requests.get(
    url=list_of_nbastats_apis["player_general_traditional_totals"], headers=headers
).json()

# pretty printing data
pretty_data = json.dumps(response, indent=4)
print(pretty_data)

In [ ]:
# pulling the headers we want to experiment with
stat_header = response["resultSets"][0]["headers"]
print(stat_header)

In [ ]:
# pulling just the data we want
body_data = response["resultSets"][0]["rowSet"]
print(body_data)

In [ ]:
# looping over data to insert into table
player_general_traditional_totals_table = []
for row in body_data:
    player_general_traditional_totals_table.append(
        [
            "2019-20",  # this is key, need this to join and sort by seasons
            row[0],
            row[1],
            row[2],
            row[3],
            row[4],
            row[5],
            row[6],
            row[7],
            row[8],
            row[9],
            row[10],
            row[11],
            row[12],
            row[13],
            row[14],
            row[15],
            row[16],
            row[17],
            row[18],
            row[19],
            row[20],
            row[21],
            row[22],
            row[23],
            row[24],
            row[25],
            row[26],
            row[27],
            row[28],
            row[29],
            row[30],
            row[31],
            row[32],
            row[33],
            # gp_rank=row[34],
            # w_rank=row[35],
            # l_rank=row[36],
            # w_pct_rank=row[37],
            # min_rank=row[38],
            # fgm_rank=row[39],
            # fga_rank=row[40],
            # fg_pct_rank=row[41],
            # fg3m_rank=row[42],
            # fg3a_rank=row[43],
            # fg3_pct_rank=row[44],
            # ftm_rank=row[45],
            # fta_rank=row[46],
            # ft_pct_rank=row[47],
            # oreb_rank=row[48],
            # dreb_rank=row[49],
            # reb_rank=row[50],
            # ast_rank=row[51],
            # tov_rank=row[52],
            # stl_rank=row[53],
            # blk_rank=row[54],
            # blka_rank=row[55],
            # pf_rank=row[56],
            # pfd_rank=row[57],
            # pts_rank=row[58],
            # plus_minus_rank=row[59],
            # nba_fantasy_points_rank=row[60],
            # dd2_rank=row[61],
            # td3_rank=row[62],
            row[63],
            row[64],
        ]
    )

print(player_general_traditional_totals_table)

In [ ]:
# Converting a List to DataFrame

# List of columns
columns = [
    "season_id",
    "player_id",
    "player_name",
    "team_id",
    "team_abbreviation",
    "age",
    "gp",
    "w",
    "l",
    "w_pct",
    "min",
    "fgm",
    "fga",
    "fg_pct",
    "fg3m",
    "fg3a",
    "fg3_pct",
    "ftm",
    "fta",
    "ft_pct",
    "oreb",
    "dreb",
    "reb",
    "ast",
    "tov",
    "stl",
    "blk",
    "blka",
    "pf",
    "pfd",
    "pts",
    "plus_minus",
    "nba_fantasy_points",
    "dd2",
    "td3",
    "cfid",
    "cfparams",
]

player_general_traditional_totals_df = pd.DataFrame(
    data=player_general_traditional_totals_table, columns=columns
)
player_general_traditional_totals_df.head()

In [ ]:
# Save to CSV
player_general_traditional_totals_df.to_csv(
    "../../data/interim/player-general-traditional.csv"
)

In [ ]:
# for x in master_header_list:
#     print(len(x))


# def non_match_elements(list_a, list_b):
#     non_match = []
#     for i in list_a:
#         if i not in list_b:
#             non_match.append(i)
#     return non_match


# non_match_head = non_match_elements(master_header_list[0], master_header_list[1])
# print(f"No match elements: {non_match_head}")
# print(master_header_list[0])
# non_match_tail = non_match_elements(master_header_list[1], master_header_list[0])
# print(f"No match elements: {non_match_tail}")
# print(master_header_list[1])